<div align="center">
<h1>Prédiction du prix des billets d'avion</h1>
</div>

L'objectif de ce projet est de prédire le prix des billets d'avion en fonction de diverses caractéristiques telles que la date de réservation, la durée du vol, le nombre d'escales, etc. Nous allons effectuer une analyse exploratoire des données (EDA) pour comprendre les relations entre les différentes variables et le prix des billets. Ensuite, nous construirons un modèle prédictif pour estimer le prix des billets en fonction des caractéristiques disponibles.

# Index

- [**Les données**](#donnees)
- [**Analyse technique**](#analyse-technique-du-jeu-de-donnees)
  - [**Analyse univariée**](#analyse-univariee)
  - [**Analyse multivariée**](#analyse-multivariee)
- [**Formuler et tester des hypothèses**](#formuler-et-tester-des-hypotheses)
- [**Preprocessing**](#preprocessing)
- [**Modélisation**](#Modelisation)
- [**Évaluation finale du modèle**](#evaluation-finale-du-modele)

# Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold

# Données

Le dataset provient de Kaggle, et est disponible [ICI](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction), c'est le dataset `Clean_Dataset.csv`. 

L'outil de scrapping Octoparse a été utilisé pour extraire les données du site web `Ease my trip`. Les données ont été collectées en deux parties : une pour les billets de classe économique et une autre pour les billets de classe affaires. 

Un total de `300.261` réservations de vol distinctes a été extrait du site, collectées sur une période de 50 jours, du 11 février au 31 mars 2022 et couvrant des vols programmés entre les 6 plus grandes villes d'Inde.
Le dataset à télécharger est une version nettoyée et pré-traitée du dataset brut.


On utilise `read_csv` de pandas pour le lire.

In [ ]:
original_df = pd.read_csv("Clean_Dataset.csv", sep=",", index_col=0)

In [ ]:
original_df.shape

In [ ]:
original_df.head(10)

**Description des colonnes :**

- airline : Nom de la compagnie aérienne.
- flight : Code du vol.
- source_city : Ville de départ.
- departure_time : Heure de départ (catégorisée: Morning, Afternoon, ...).
- stops : Nombre d'escales (zero, one, two_or_more).
- arrival_time : Heure d'arrivée (catégorisée: Morning, Afternoon, ...).
- destination_city : Ville d'arrivée.
- class : Classe du billet (économique ou business).
- duration : Durée du vol (en heures).
- days_left : Nombre de jours entre la date de réservation et la date du vol.
- price : Prix du billet (variable cible) en Roupie indienne.

Chaque ligne du dataset représente donc un vol avec ses caractéristiques et son prix associé.

In [ ]:
original_df.info()

---

# Analyse technique du jeu de données

Avant toute analyse statistique, il faut comprendre **la nature exacte des variables** que nous avons entre les mains.  
Notre objectif ici est double :

1. **Identifier le type de chaque variable**  
   - Variable discrète ?  
   - Variable continue ?  
   - Date ? Texte libre ?  
   - Fausses quantités ? (ex : durée exprimée en `String`)

2. **Identifier ce que l’on peut faire avec chacune d’elles**  
   → Est-ce qu’on peut calculer des statistiques ?  
   → Est-ce qu’on peut compter des catégories ?  
   → Est-ce qu’on doit transformer des colonnes avant toute analyse ?

Cette étape sert à déterminer ce qui est faisable en analyse univariée et multivariée.

On commence donc par lister toutes les colonnes et examiner leur contenu.

In [ ]:
# Aperçu des colonnes et des premiers éléments de chaque variable
original_df.head()

In [ ]:
original_df.dtypes

### Classification des variables (discrète / continue / autre)

À partir du type réel (`object`, `int`, etc.) et de l'observation des valeurs, on va classifier les variables de façon statistique.

- **Airline** → Discrète (catégorielle)
- **Flight** → Discrète (catégorielle)
- **Source_city** → Discrète (catégorielle)
- **Departure_time** → Discrète (catégorielle ordinale)
- **Stops** → Discrète (numérique ordinale)
- **Arrival_time** → Discrète (catégorielle ordinale)
- **Destination_city** → Discrète (catégorielle)
- **Class** → Discrète (catégorielle)
- **Duration** → Continue (numérique)
- **Days_left** → Continue (numérique)
- **Price** → Continue (numérique) - Variable cible

On vérifie maintenant l’unicité des catégories et une statistique simple sur les variables continues.

In [ ]:
discrete_cols = ["airline", "flight", "source_city", "departure_time", "stops", "arrival_time", "destination_city", "class"]
continues_cols = ["duration", "days_left", "price"]

In [ ]:
# Comptage rapide des catégories pour les variables discrètes
for col in discrete_cols:
    print(f"\n--- {col} ---")
    print(original_df[col].value_counts().head())

## Analyse Univariée

Maintenant que les variables ont été classées, on commence l’analyse statistique variable par variable.

L’objectif est ici :

- Pour les **variables discrètes** : lister les effectifs et visualiser les catégories majoritaires/minoritaires.  
- Pour les **variables continues** : voir la distribution, les valeurs extrêmes, les asymétries, etc.

On commence par l’analyse des **variables discrètes** du dataset.

In [ ]:
def univ_analysis_discrete(col_name, max_unique=20):
    print(f"\n===== {col_name} =====")
    display(original_df[col_name].value_counts())
    # Bar plot
    plt.figure(figsize=(10, 5))
    if original_df[col_name].nunique() <= max_unique:
        original_df[col_name].value_counts().plot(kind='bar')
        plt.title(f"Distribution of {col_name}")
        plt.xlabel(col_name)
        plt.ylabel("Count")
        plt.xticks(rotation=45)
    else:
        original_df[col_name].value_counts().nlargest(max_unique).plot(kind='bar')
        plt.title(f"Top {max_unique} categories of {col_name}")
        plt.xlabel(col_name)
        plt.ylabel("Count")
        plt.xticks(rotation=45)
    plt.show()

### Airline

In [ ]:
univ_analysis_discrete("airline")

- Variable **discrète**, à forte part de marché inégale.  
- Impact probable sur le prix → à analyser dans la partie *multivariée*.

### Source_city

In [ ]:
univ_analysis_discrete("source_city")

 - Peut fortement influencer le prix selon la destination → à explorer plus loin.

### Departure_time

In [ ]:
univ_analysis_discrete("departure_time")

- Très peu de vols tardifs

### Stops

In [ ]:
univ_analysis_discrete("stops")

- Variable **discrète ordinale** (zero/One/two_or_more stops). 
- À transformer en variable numérique ordonnée pour la modélisation.

### Arrival_time

In [ ]:
univ_analysis_discrete("arrival_time")

- Beaucoup de vols arrivent tard le soir ou la nuit.

### Destination_city

In [ ]:
univ_analysis_discrete("destination_city")

### Class

In [ ]:
univ_analysis_discrete("class")

- À peu près deux fois plus de billets en classe économique qu'en classe business.

On poursuit avec l’analyse des **variables continues** du dataset.

In [ ]:
def univ_analysis_continuous(col_name, nbins=30):
    print(f"\n===== {col_name} =====")
    display(original_df[col_name].describe())
    # box plot and histogram
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    original_df.boxplot(column=col_name)
    plt.title(f"Box plot pour {col_name}")
    plt.subplot(1, 2, 2)
    original_df[col_name].hist(bins=nbins)
    plt.title(f"Histograme pour {col_name}")
    plt.xlabel(col_name)
    plt.ylabel("Fréquence")
    plt.show()

### Duration

In [ ]:
univ_analysis_continuous("duration", nbins=120)

### Days_left

In [ ]:
univ_analysis_continuous("days_left", nbins=20)

### Price

In [ ]:
univ_analysis_continuous("price")

- La moyenne du prix est de 20800 Roupies, mais la médiane est à 7400 Roupies, ce qui indique une distribution asymétrique avec une longue queue à droite (quelques billets très chers). C'est ce qu'on observe dans l'histogramme.
- Les deux groupes de prix correspondent en fait aux classes économique et business.

## Analyse multivariée

On s'interesse maintenant aux relations entre les différentes variables. Notamment comment les variables influencent le prix des billets d'avion. 

### Relation entre la durée du vol et le prix

In [ ]:
# On crée des tranches (bins)
original_df["duration_bin"] = pd.cut(original_df["duration"], bins=[0,5,10,20,40,60])

plt.figure(figsize=(10, 6))
sns.boxplot(data=original_df, x="duration_bin", y="price")
plt.title("Prix par tranche de duration")
plt.xlabel("Tranche de durée de vol")
plt.ylabel("Prix")
plt.show()

Ici, les prix moyens semblent varier positivement avec la durée du vol.

### Relation entre le prix et la compagnie aérienne

In [ ]:
# Relation entre airline et price
plt.figure(figsize=(12, 6))
sns.boxplot(data=original_df, x="airline", y="price")
plt.title("Relation entre la compagnie aérienne et le prix")
plt.xlabel("Compagnie aérienne")
plt.ylabel("Prix (Roupies)")
plt.xticks(rotation=45)
plt.show()

On remarque des valeurs plus élevées pour certaines compagnies aériennes, ici Vistara et Air India, ce qui suggère que la compagnie propose un service premium par exemple.


La compagnie aérienne **semble** avoir un impact significatif sur le prix des billets d'avion.

### Relation entre le nombre d'escales et le prix

In [ ]:
# Relation entre stops et price
plt.figure(figsize=(10, 6))
sns.boxplot(data=original_df, x="stops", y="price")
plt.title("Relation entre le nombre d'escales et le prix")
plt.xlabel("Nombre d'escales")
plt.ylabel("Prix (Roupies)")
plt.show()

En moyenne, le prix est plus élevé pour les vols avec au moins une escale. Cependant, il y a une grande variabilité des prix pour chaque catégorie d'escales, indiquant que d'autres facteurs influencent également le prix des billets.

### Relation entre le nombre de jours entre la réservation et le vol et le prix

In [ ]:
# On crée des tranches (bins)
original_df["days_bin"] = pd.cut(original_df["days_left"], bins=[0,5,10,20,40,60])

plt.figure(figsize=(10, 6))
sns.boxplot(data=original_df, x="days_bin", y="price")
plt.title("Prix par tranche de days_left")
plt.xlabel("Tranche de jours avant le vol")
plt.ylabel("Prix")
plt.show()

La moyenne des prix tend à diminuer avec le nombre de jours entre la réservation et le vol. Réserver plus tôt **semble** être associé à des prix plus bas.

---

# Formuler et tester des hypothèses

Après l'analyse technique, univariée et multivariée, on va maintenant formuler des **hypothèses statistiques** et les **tester**. L’idée est de partir de nos observations précédentes et de les transformer en hypothèses, que l’on va ensuite valider ou non à l’aide de tests statistiques.

On prendra pour ces tests un seuil de signification classique de 5% (α = 0.05).

In [ ]:
alpha = 0.05  # seuil de significativité (5%)

À partir des graphiques et des statistiques calculées précédemment, on retient les observations suivantes :


## 1. Durée du vol vs Prix — Test de corrélation de Pearson

**Question :** les vols plus longs sont-ils associés à des prix plus élevés ?

On teste ici l’existence d’une **corrélation linéaire** entre :

- `duration` (variable continue)
- `price` (variable continue)

**Hypothèses :**

- $H_0$ : la corrélation linéaire entre `duration` et `price` est nulle.  
- $H_1$ : la corrélation linéaire entre `duration` et `price` est non nulle.

On utilise le **test de Pearson**.

In [ ]:
# Test de corrélation de Pearson entre duration et price
corr_duration_price, p_value_duration_price = stats.pearsonr(
    original_df["duration"],
    original_df["price"]
)

print("Corrélation (duration, price) :", corr_duration_price)
print("p-value :", p_value_duration_price)

if p_value_duration_price < alpha:
    print(f"→ p-value < {alpha} : on rejette H0.")
    print("Conclusion : il existe une corrélation linéaire significative entre la durée du vol et le prix.")
else:
    print(f"→ p-value ≥ {alpha} : on ne rejette pas H0.")
    print("Conclusion : on ne met pas en évidence de corrélation linéaire significative entre la durée du vol et le prix.")

## 2. Compagnie aérienne vs Prix — Test ANOVA

**Question :** les différentes compagnies pratiquent-elles en moyenne le même niveau de prix ?

- Variable discrète : `airline` (plusieurs catégories)
- Variable continue : `price`

Pour garder des groupes suffisamment remplis, on limite l’étude aux **compagnies les plus fréquentes** dans le dataset.

**Hypothèses :**

- $H_0$ : le prix moyen des billets est le même pour toutes les compagnies considérées.  
- $H_1$ : au moins une des compagnies a un prix moyen différent des autres.

On utilise un **test ANOVA (f\_oneway)**.

In [ ]:
# On sélectionne les N compagnies les plus fréquentes
N = 5
top_airlines = original_df["airline"].value_counts().nlargest(N).index
df_top_airlines = original_df[original_df["airline"].isin(top_airlines)]

# Construction des groupes de prix par compagnie
groups_price_by_airline = [
    df_top_airlines[df_top_airlines["airline"] == airline]["price"].values
    for airline in top_airlines
]

# Test ANOVA
f_stat_airline, p_value_airline = stats.f_oneway(*groups_price_by_airline)

print("Compagnies étudiées :", list(top_airlines))
print("Statistique F (airline vs price) :", f_stat_airline)
print("p-value :", p_value_airline)

if p_value_airline < alpha:
    print(f"→ p-value < {alpha} : on rejette H0.")
    print("Conclusion : le prix moyen n’est pas le même pour toutes ces compagnies (au moins une se distingue).")
else:
    print(f"→ p-value ≥ {alpha} : on ne rejette pas H0.")
    print("Conclusion : on n’a pas de preuve suffisante pour dire que le prix moyen diffère entre ces compagnies.")

## 3. Nombre d’escales vs Prix — Test ANOVA

**Question :** le prix moyen est-il identique quel que soit le nombre d’escales ?

- Variable discrète : `stops`
- Variable continue : `price`

**Hypothèses :**

- $H_0$ : le prix moyen des billets est le même pour tous les niveaux d’escales.  
- $H_1$ : le prix moyen des billets dépend du nombre d’escales.

On utilise un **test ANOVA** sur les groupes définis par `stops`.

In [ ]:
# On s'assure que 'stops' est de type "catégoriel"/str pour bien définir les groupes et les ordonner
stops_unique = sorted(original_df["stops"].unique())
print("Valeurs de 'stops' :", stops_unique)

groups_price_by_stops = [
    original_df[original_df["stops"] == s]["price"].values
    for s in stops_unique
]

# Test ANOVA
f_stat_stops, p_value_stops = stats.f_oneway(*groups_price_by_stops)

print("Statistique F (stops vs price) :", f_stat_stops)
print("p-value :", p_value_stops)

if p_value_stops < alpha:
    print(f"→ p-value < {alpha} : on rejette H0.")
    print("Conclusion : le prix moyen des billets dépend du nombre d’escales.")
else:
    print(f"→ p-value ≥ {alpha} : on ne rejette pas H0.")
    print("Conclusion : on n’a pas de preuve suffisante pour dire que le prix moyen varie selon le nombre d’escales.")

## 4. Nombre de jours avant le vol vs Prix — Test de corrélation de Pearson

**Question :** Réserver plus tôt est-il associé à des prix plus bas ?

On étudie la relation entre :

- `days_left` (nombre de jours entre la réservation et le vol)
- `price` (prix du billet)

**Hypothèses :**

- $H_0$ : il n’existe pas de corrélation linéaire entre `days_left` et `price`.  
- $H_1$ : il existe une corrélation linéaire entre `days_left` et `price` (par exemple, prix plus faible lorsque `days_left` augmente).

On utilise le **test de corrélation de Pearson**.

In [ ]:
# Test de corrélation de Pearson entre days_left et price
corr_days_price, p_value_days_price = stats.pearsonr(
    original_df["days_left"],
    original_df["price"]
)

print("Corrélation (days_left, price) :", corr_days_price)
print("p-value :", p_value_days_price)

if p_value_days_price < alpha:
    print(f"→ p-value < {alpha} : on rejette H0.")
    print("Conclusion : il existe une corrélation linéaire significative entre le nombre de jours avant le vol et le prix.")
    print("Interprétation possible : réserver plus tôt est effectivement associé à des prix différents (souvent plus bas).")
else:
    print(f"→ p-value ≥ {alpha} : on ne rejette pas H0.")
    print("Conclusion : on ne met pas en évidence de corrélation linéaire significative entre days_left et price.")

---

# Preprocessing

Après l'analyse technique, univariée, multivariée, puis les tests d’hypothèses, on va maintenant préparer les données pour la modélisation.

L’objectif du preprocessing est de transformer notre dataset brut en un jeu de données exploitable par un modèle de machine learning :

- Nettoyer les valeurs aberrantes
- Encoder les variables catégorielles
- Créer éventuellement quelques features supplémentaires
- Normaliser les variables numériques
- Préparer une pipeline scikit-learn réutilisable

On part d'une copie du dataset initial.

In [32]:
df = original_df.copy()

On commence par compter le nombre de valeurs manquantes par colonne.

In [34]:
print("Nombre de NaNs par colonne :")
print(df.isna().sum())

Nombre de NaNs par colonne :
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
duration_bin        0
days_bin            0
dtype: int64


## Encodage simple de `stops` et `class`

Avant de construire la pipeline scikit-learn, on va encoder deux variables importantes :

- `stops` : variable ordinale (zero, one, two_or_more) → 0 / 1 / 2
- `class` : Economy / Business → 0 / 1

Ces nouvelles colonnes numériques (`stops_num` et `class_num`) pourront ensuite être normalisées et utilisées comme de vraies variables numériques dans le modèle.

In [39]:
# Encodage ordinal de stops
mapping_stops = {"zero": 0, "one": 1, "two_or_more": 2}
df["stops_num"] = df["stops"].map(mapping_stops)
print(df[["stops", "stops_num"]].head())

  stops  stops_num
0  zero          0
1  zero          0
2  zero          0
3  zero          0
4  zero          0


In [40]:
# Encodage binaire de class
mapping_class = {"Economy": 0, "Business": 1}
df["class_num"] = df["class"].map(mapping_class)
print(df[["class", "class_num"]].head())

     class  class_num
0  Economy          0
1  Economy          0
2  Economy          0
3  Economy          0
4  Economy          0


## Séparation features / target et train/test split

- **Target** : `price`
- **Features** : toutes les autres colonnes utiles.

On va :
1. Supprimer la colonne `flight` (identifiant peu informatif pour le modèle).
2. Supprimer les anciennes versions catégorielles de `stops` et `class` (on garde les versions numériques).
3. Séparer en X (features) et y (target).
4. Faire un `train_test_split` avec 20% des données en test.

In [41]:
# On enlève la colonne 'flight' (identifiant) et les anciennes versions de stops/class
df_model = df.drop(columns=["flight", "stops", "class"])

In [42]:
# Séparation features / target
X = df_model.drop(columns=["price"])
y = df_model["price"]

In [43]:
# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train.shape, X_test.shape

((240122, 11), (60031, 11))

## Définir les colonnes numériques et catégorielles (après le préprocessing simple)

On sépare les features en deux groupes :

- **Features numériques** : 
  - `duration`
  - `days_left`
  - `stops_num`
  - `class_num`

Ces variables seront imputées (si besoin) puis **standardisées** (mean=0, std=1).

- **Features catégorielles** (nominales, sans ordre) :
  - `airline`
  - `source_city`
  - `departure_time`
  - `arrival_time`
  - `destination_city`

Ces variables seront **encodées en one-hot** via `OneHotEncoder` (avec `drop="first"` pour éviter la multicolinéarité).

In [44]:
# Colonnes numériques
numeric_features = ["duration", "days_left", "stops_num", "class_num"]

In [45]:
# Colonnes catégorielles
categorical_features = ["airline", "source_city", "departure_time", 
                        "arrival_time", "destination_city"]

In [46]:
# Pipeline pour les variables numériques : imputation + standardisation
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

In [47]:
# Pipeline pour les variables catégorielles : imputation + one-hot encoding
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="first")),
])

In [48]:
# ColumnTransformer qui applique le bon traitement à chaque type de variable
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


## Préprocesseur prêt pour la modélisation

L’objet `preprocessor` contient désormais toute la logique de preprocessing :

1. **Nettoyage** (via les imputers)
2. **Encodage** des variables catégorielles (One-Hot)
3. **Normalisation** des variables numériques (StandardScaler)

Dans la partie suivante (*Modélisation*), on pourra facilement construire un modèle complet en créant une `Pipeline` :

- étape 1 : `("preprocessor", preprocessor)`
- étape 2 : `("regressor", <modèle_de_régression>)`

Par exemple, une régression linéaire, une régression Ridge, un RandomForestRegressor, etc.

Cette approche permet :
- d’éviter les fuites de données (data leakage),
- de réappliquer exactement les mêmes transformations sur les données de test,
- et plus tard, sur de nouvelles données en production.

---

# Modélisation

Nous avons maintenant :

- une cible : **`price`** (variable continue),
- un jeu de features prétraitées (via l’objet `preprocessor`),
- une séparation **train / test** : `X_train`, `X_test`, `y_train`, `y_test`.

Nous sommes donc dans un cadre d’**apprentissage supervisé** pour un problème de **régression**.

Dans cette section, nous allons :

1. Construire une **baseline** avec un modèle naïf (DummyRegressor)
2. Entraîner une **régression linéaire** (LinearRegression)
3. Ajouter une **régression Ridge** (modèle linéaire régularisé)
4. Tester un **modèle non linéaire** (RandomForestRegressor)
5. Comparer les performances de tous ces modèles
6. Illustrer le réglage d’**hyperparamètres** avec `GridSearchCV`

In [66]:
# Petite fonction utilitaire pour évaluer un modèle
def evaluate_model(
    name,
    model,
    X_train,
    y_train,
    X_test,
    y_test,
    results_dict,
    cv=5,
):
    """
    Évalue un modèle avec validation croisée sur le train,
    puis refit sur tout le train et évalue une fois sur le test.

    Paramètres
    ----------
    name : str
        Nom du modèle (pour l'affichage et le dictionnaire de résultats).
    model : estimator / Pipeline
        Modèle ou Pipeline scikit-learn (déjà construit).
    X_train, y_train : array-like
        Données d'entraînement.
    X_test, y_test : array-like
        Données de test.
    results_dict : dict
        Dictionnaire dans lequel on stocke les résultats.
    cv : int
        Nombre de folds pour la validation croisée.
    """

    # ----- 1) Validation croisée sur le train -----
    cv_results = cross_validate(
        estimator=model,
        X=X_train,
        y=y_train,
        cv=KFold(n_splits=cv, shuffle=True, random_state=42),
        scoring={
            "mae": "neg_mean_absolute_error",
            "rmse": "neg_root_mean_squared_error",
            "r2": "r2",
        },
        n_jobs=-1,
        return_train_score=False,
    )

    # Moyennes (on remet les signes positifs pour MAE / RMSE)
    cv_mae_mean = -cv_results["test_mae"].mean()
    cv_mae_std  =  cv_results["test_mae"].std()

    cv_rmse_mean = -cv_results["test_rmse"].mean()
    cv_rmse_std  =  cv_results["test_rmse"].std()

    cv_r2_mean = cv_results["test_r2"].mean()
    cv_r2_std  = cv_results["test_r2"].std()

    print(f"=== {name} ===")
    print(f"Validation croisée ({cv}-fold) sur le train :")
    print(f"  MAE  (mean ± std) : {cv_mae_mean:.2f} ± {cv_mae_std:.2f}")
    print(f"  RMSE (mean ± std) : {cv_rmse_mean:.2f} ± {cv_rmse_std:.2f}")
    print(f"  R²   (mean ± std) : {cv_r2_mean:.4f} ± {cv_r2_std:.4f}")

    # ----- 2) Refit sur tout le train -----
    model.fit(X_train, y_train)

    # ----- 3) Évaluation finale sur le test -----
    y_test_pred = model.predict(X_test)

    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_rmse = root_mean_squared_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    print("Évaluation finale sur le test :")
    print(f"  Test MAE  : {test_mae:.2f}")
    print(f"  Test RMSE : {test_rmse:.2f}")
    print(f"  Test R²   : {test_r2:.4f}")
    print()

    results_dict[name] = {
        "cv_mae_mean": cv_mae_mean,
        "cv_mae_std": cv_mae_std,
        "cv_rmse_mean": cv_rmse_mean,
        "cv_rmse_std": cv_rmse_std,
        "cv_r2_mean": cv_r2_mean,
        "cv_r2_std": cv_r2_std,
        "test_mae": test_mae,
        "test_rmse": test_rmse,
        "test_r2": test_r2,
    }

In [67]:
# Dictionnaire pour stocker les résultats de tous les modèles
results = {}

## 1. Baseline — DummyRegressor

Avant d’entraîner de “vrais” modèles de Machine Learning, on commence par une **baseline naïve**.

On utilise ici un `DummyRegressor` qui prédit toujours la **moyenne** des prix observés sur le jeu d’entraînement.

Ce modèle :
- ne regarde pas les features,
- sert uniquement de **référence** : un bon modèle doit faire mieux que lui.

In [68]:
dummy_model = DummyRegressor(strategy="mean")

evaluate_model(
    name="DummyRegressor (mean)",
    model=dummy_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    results_dict=results,
)

=== DummyRegressor (mean) ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 19757.22 ± 35.06
  RMSE (mean ± std) : 22696.11 ± 47.23
  R²   (mean ± std) : -0.0000 ± 0.0000
Évaluation finale sur le test :
  Test MAE  : 19768.69
  Test RMSE : 22704.24
  Test R²   : -0.0000



## 2. Régression linéaire — `LinearRegression`

On passe maintenant à un premier “vrai” modèle de Machine Learning : la **régression linéaire**.

Pour l’utiliser proprement avec nos données :

- On intègre le **preprocessing** (encodage + normalisation) dans une `Pipeline`.
- On ajoute ensuite un estimateur `LinearRegression`.

La `Pipeline` permet de :

- `fit` le préprocesseur uniquement sur le **train**,
- appliquer exactement les mêmes transformations sur le **test**,
- éviter les fuites de données (data leakage).

In [69]:
linreg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

evaluate_model(
    name="LinearRegression",
    model=linreg_pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    results_dict=results,
)

=== LinearRegression ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 4526.59 ± 14.72
  RMSE (mean ± std) : 6809.86 ± 26.06
  R²   (mean ± std) : 0.9100 ± 0.0008
Évaluation finale sur le test :
  Test MAE  : 4500.71
  Test RMSE : 6814.94
  Test R²   : 0.9099



## 3. Régression linéaire régularisée — `Ridge`

La régression linéaire classique peut parfois :

- mal généraliser,
- être sensible aux colinéarités entre features.

La **régression Ridge** ajoute une **pénalité L2** sur la taille des coefficients, contrôlée par un hyperparamètre `alpha` :

- `alpha` petit → peu de régularisation (proche de LinearRegression)
- `alpha` grand → coefficients plus “petits”, modèle plus régularisé

On va commencer par tester quelques valeurs de `alpha` “à la main”, sans GridSearch, pour voir l’impact sur les performances.

In [70]:
alphas = [0.1, 1.0, 20.0]

for a in alphas:
    ridge_pipeline = Pipeline(steps=[
        ("preprocessor", preprocessor),
        ("model", Ridge(alpha=a, random_state=42))
    ])
    
    evaluate_model(
        name=f"Ridge (alpha={a})",
        model=ridge_pipeline,
        X_train=X_train,
        y_train=y_train,
        X_test=X_test,
        y_test=y_test,
        results_dict=results,
    )

=== Ridge (alpha=0.1) ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 4526.56 ± 14.73
  RMSE (mean ± std) : 6809.86 ± 26.07
  R²   (mean ± std) : 0.9100 ± 0.0008
Évaluation finale sur le test :
  Test MAE  : 4500.74
  Test RMSE : 6814.97
  Test R²   : 0.9099

=== Ridge (alpha=1.0) ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 4526.57 ± 14.72
  RMSE (mean ± std) : 6809.86 ± 26.07
  R²   (mean ± std) : 0.9100 ± 0.0008
Évaluation finale sur le test :
  Test MAE  : 4500.71
  Test RMSE : 6814.96
  Test R²   : 0.9099

=== Ridge (alpha=20.0) ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 4526.34 ± 14.71
  RMSE (mean ± std) : 6809.86 ± 26.06
  R²   (mean ± std) : 0.9100 ± 0.0008
Évaluation finale sur le test :
  Test MAE  : 4500.51
  Test RMSE : 6814.97
  Test R²   : 0.9099



## 4. Modèle non linéaire — `RandomForestRegressor`

Pour capturer des relations **non linéaires** entre les variables et le prix, on peut utiliser une **forêt aléatoire de régression** (`RandomForestRegressor`).

Principe :

- on entraîne plusieurs **arbres de décision** sur des sous-échantillons des données,
- on moyenne leurs prédictions,
- le modèle est souvent robuste et performant sur des données tabulaires.

Hyperparamètres importants :

- `n_estimators` : nombre d’arbres,
- `max_depth` : profondeur maximale des arbres,
- `min_samples_leaf` : nombre minimum d’échantillons dans une feuille.

On commence avec des valeurs “raisonnables” et les hyperparamètres par défaut pour les autres. 

Temps d’entraînement plus long, mais souvent de bonnes performances (environ 5 minutes).

In [72]:
rf_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=50,
        max_depth=None,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    ))
])

evaluate_model(
    name="RandomForestRegressor",
    model=rf_pipeline,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    results_dict=results,
)

=== RandomForestRegressor ===
Validation croisée (5-fold) sur le train :
  MAE  (mean ± std) : 1119.36 ± 4.20
  RMSE (mean ± std) : 2790.69 ± 14.47
  R²   (mean ± std) : 0.9849 ± 0.0002
Évaluation finale sur le test :
  Test MAE  : 1090.26
  Test RMSE : 2812.55
  Test R²   : 0.9847



## 5. Comparaison des modèles

On regroupe maintenant les résultats de tous les modèles testés dans un tableau récapitulatif.

On compare en particulier :

- la **baseline** (DummyRegressor),
- la **régression linéaire**,
- la **régression Ridge** pour différentes valeurs de `alpha`,
- la **Random Forest**.

Les métriques regardées :

- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- R² (coefficient de détermination)

L’objectif est de voir :

- quels modèles généralisent le mieux (bon score sur le test),
- s’il y a de l’overfitting (écart train/test important),
- et lequel on retiendrait comme modèle “principal” pour ce projet.

In [73]:
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by="test_rmse")
results_df

,cv_mae_mean,cv_mae_std,cv_rmse_mean,cv_rmse_std,cv_r2_mean,cv_r2_std,test_mae,test_rmse,test_r2
RandomForestRegressor,1119.363841,4.199134,2790.693805,14.471564,0.984880,0.000211,1090.257087,2812.546702,9.846543e-01
LinearRegression,4526.585422,14.724885,6809.857284,26.062729,0.909969,0.000781,4500.712404,6814.935625,9.099031e-01
Ridge (alpha=1.0),4526.571391,14.721204,6809.859572,26.065448,0.909969,0.000780,4500.708620,6814.961331,9.099024e-01
Ridge (alpha=20.0),4526.339150,14.705239,6809.860505,26.061420,0.909969,0.000780,4500.513976,6814.969366,9.099022e-01
Ridge (alpha=0.1),4526.560218,14.726794,6809.856254,26.067886,0.909969,0.000781,4500.736732,6814.971944,9.099022e-01
DummyRegressor (mean),19757.219845,35.060714,22696.113029,47.233583,-0.000015,0.000013,19768.694246,22704.235357,-5.741994e-08


Ici, le meilleur modèle est clairement le **RandomForestRegressor**, qui capture bien les relations non linéaires et les interactions entre variables. 

On passe maintenant à l'entrainement du modèle final sur l'ensemble des données d'entrainement.

In [74]:
best_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=100,
        max_depth=None,
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1,
    ))
])

best_model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# Évaluation finale du modèle

In [76]:
y_test_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_test_pred)
rmse = root_mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

In [77]:
print(f"Test MAE  : {mae:.2f}")
print(f"Test RMSE : {rmse:.2f}")
print(f"Test R²   : {r2:.4f}")

Test MAE  : 1086.89
Test RMSE : 2801.59
Test R²   : 0.9848


On obtient les performances finales du modèle sur le jeu de test.